# IMPORT PACKAGES

In [1]:
import glob
import pandas as pd
import math
import glob
import pandas as pd
import numpy as np
from pandas import DataFrame
from numpy import matrix 
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from kerastuner.tuners import RandomSearch
import matplotlib.pyplot as plt
from numpy import matrix
import pandas as pd
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from pandas import DataFrame
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import kerastuner as kt
from pandas import DataFrame
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import callbacks
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import roc_auc_score
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.layers import (
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling2D
)
from kerastuner.tuners import Hyperband

from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
import kerastuner as kt
import seaborn as sns
sns.set_style('whitegrid')

2022-09-12 20:34:30.957577: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/luke/ws_moveit/devel/lib
2022-09-12 20:34:30.957626: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# READ MATRICES

Pearson's connection matrix of the EEG experiments of subjects who ingested Ayahuasca at the time after the psychedelic activation time can be found in: Alves, Caroline (2022): With-ayahuasca. figshare. Dataset. https://doi.org/10.6084/m9.figshare.21082513.v1

Pearson's connection matrix of the EEG experiments of subjects who ingested Ayahuasca at the time before the psychedelic activation time: Alves, Caroline (2022): No-ayahuasca. figshare. Dataset.
https://doi.org/10.6084/m9.figshare.21082531.v1

In [ ]:
## Define function to read
def pixels_from_path(file_path):
    im = open(file_path)
    test = pd.read_csv(im,sep=",", index_col=False,header=None)
    #im = im.resize(IMG_SIZE)
    np_im = np.array(test)
    #matrix of pixel RGB values
    return np_im

print("with ayahuasca")
#insert path for Pearson connectivity matrix with ayahuasca
aya = np.asarray([pixels_from_path(n) for n in glob.glob(r'PATH-ayahuasca/*csv')])
print("shape for ayahuasca matrices")
print(np.shape(aya ))
#insert path for Pearson connectivity matrix no ayahuasca
print("no ayahuasca")
normal = np.asarray([pixels_from_path(n) for n in glob.glob(r'aPATH-no-yahuasca/*csv')])
print(normal)
print("shape for no ayahuasca matrices")
print(np.shape(normal))


# PREPARING TRAIN SET AND LABELS

In [ ]:
## Preparing train set
x_train = np.concatenate([normal ,aya])
x=np.nan_to_num(x_train) 
print("shape for train set")
print(np.shape(x_train))

## Preparing test set
yes_l = [1]*len(aya)
no_l = [0]*len(normal)
labels = yes_l + no_l
#print(labels) ## the labels of the graphs
y=np.nan_to_num(labels)
print("length for test set")
print(len(y))

# DEFINE CNN MODEL AND TUNNING HYPERPARAMETERS

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]


class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        model.add(
            Conv2D(
                filters=16,
                kernel_size=3,
                activation='relu',
                input_shape=self.input_shape
            )
        )
        model.add(
            Conv2D(
                filters=16,
                activation='relu',
                kernel_size=3
            )
        )
        model.add(MaxPooling2D(pool_size=2))
        model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.01,
                max_value=0.5,
                default=0.25,
                step=0.05,
            ))
        )
        model.add(
            Conv2D(
                filters=32,
                kernel_size=3,
                activation='relu'
            )
        )
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'num_filters',
                    values=[32, 64],
                    default=64,
                ),
                activation='relu',
                kernel_size=3
            )
        )
        model.add(MaxPooling2D(pool_size=2))
        model.add(
            Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.05,
                max_value=0.5,
                default=0.25,
                step=0.05,
            ))
        )
        model.add(Flatten())
        model.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_3',
                    min_value=0.001,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='categorical_crossentropy',
            metrics=METRICS
        )
        return model


# RESAMPLING AND CALLING THE MODEL

In [ ]:
SEED = 1
NUM_CLASSES = 2  #
HYPERBAND_MAX_EPOCHS = 100
MAX_TRIALS = 100
EXECUTION_PER_TRIAL = 2
INPUT_SHAPE = (63, 63, 1) 
#X_train, X_test, Y_train, Y_test = train_test_split(x, y, stratify=y, test_size=0.10)
skfold = StratifiedKFold(n_splits=10)
for train_index, test_index in skfold.split(X_train,Y_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = X_train[train_index], X_train[test_index]
    y_train, y_test = Y_train[train_index], Y_train[test_index]
    print(np.shape(x_train))
    
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)

    #count_classes = y_test.shape[1]
    x_test = x_test.reshape(-1, 63, 63, 1)
    x_train = x_train.reshape(-1, 63, 63, 1)
    
 

    hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
    tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
   
     
    overwrite = True  

    )
    tuner.search(x_train, y_train, epochs=100,  validation_data=(x_test,y_test), verbose=0)
    best_model_rs = tuner.get_best_models(num_models=1)[0]
#tuner.results_summary()
# Retrieve the best model.
    best_model_rs = tuner.get_best_models(num_models=1)[0]
    history_rs=best_model_rs.fit(x_train,y_train, epochs=100, validation_data=(x_test,y_test))
    print(best_model_rs.evaluate(x_train, y_train))


# PRINT RESULTS

In [ ]:
print("Print results for train set")
best_model_rs.evaluate(x_train, y_train)
X_test = X_test.reshape(-1, 63, 63, 1)
Y_test = to_categorical(Y_test)
print("Print results for test set")
best_model_rs.evaluate(X_test, Y_test)

# PLOT ROC CURVE

In [ ]:
y_score= best_model_rs.predict(X_test)
# Plot linewidth.
lw = 2
n_classes=2
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(Y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(8,5))
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='slategray', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['purple', 'lightseagreen'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--',color='#cb416b', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.annotate(' Random Guess',(.5,.48),color='#cb416b')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
#plt.title('Receiver operating characteristic obtained by the model found by the Random Search algorithm \n for the Granger Connectivity Matrices for Alzheimer database ')
plt.legend(loc="lower right")
plt.show()


# PLOT CONFUSION MATRIX

In [ ]:
y_score= best_model_rs.predict(X_test)
y_score=np.argmax(y_score, axis=1)
Y_test=np.argmax(Y_test, axis=1)
cm = metrics.confusion_matrix(y_score, Y_test)
cm_sum = np.sum(cm, axis=1)
cm_perc = cm / cm_sum.astype(float) * 100
annot = np.empty_like(cm).astype(str)
nrows, ncols = cm.shape
for i in range(nrows):
    for j in range(ncols):
        c = cm[i, j]
        p = cm_perc[i, j]
        if i == j:
            s = cm_sum[i]
            annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
        elif c == 0:
            annot[i, j] = ''
        else:
            annot[i, j] = '%.1f%%\n%d' % (p, c)
cm = pd.DataFrame(cm)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
#fig, ax = plt.subplots(figsize=figsize)
sns.heatmap(cm, annot=annot, fmt='',cmap='rocket_r')

# PLOT LEARNING CURVE

In [ ]:
plt.figure(figsize=(8,5))
acc = history_rs.history['accuracy']
val_acc =history_rs.history['val_accuracy']
loss = history_rs.history['loss']
val_loss = history_rs.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()